In [1]:
##########################################
#Examples for different configurations:
#
# Assume example data set:
# - Patient1
#   - pat1_TP1_MR
#   - pat1_TP2_MR
#   - pat1_TP1_CT
# - Patient2
#   - pat2_TP1_MR1
#   - pat2_TP1_MR2
#   - pat2_TP2_MR1
#   - pat2_TP1_CT
#   - pat2_TP2_CT

In [2]:
###############################################################################
# Imports
###############################################################################
import os

import avid.common.workflow as workflow

from avid.selectors import ActionTagSelector as ATS
from avid.actions.pythonAction import PythonNaryBatchAction as naryPythonAction, PythonNaryBatchActionV2

from avid.linkers import CaseLinker, TimePointLinker, FractionLinker
from avid.splitter import SingleSplitter, CaseSplitter, KeyValueSplitter

In [3]:
###############################################################################
# Define ActionTagSelectors
###############################################################################
mr_image_selector = ATS('MR')
ct_image_selector = ATS('CT')
mask_selector = ATS('Seg')

In [4]:
###############################################################################
# Define callable
###############################################################################
def my_function(outputs, **kwargs):
    '''
        Simple print-callable that outputs the combinations of input artifacts.
    '''
    keys = kwargs.keys()
    print_output = 'output: {'
    
    for i, key in enumerate(keys):
        if i != 0:
            print_output += ', '
        print_output += '{}:{}'.format(key, [os.path.basename(mr) for mr in kwargs[key]])
    print_output += '}'
    
    print(print_output)
    with open(outputs[0], "w") as ofile:
        ofile.write(str(kwargs))

In [7]:
###############################################################################
# Initialize session
###############################################################################
session =  workflow.initSession(bootstrapArtefacts=os.path.join(os.getcwd(),'output', 'example.avid'),
                                sessionPath=os.path.join(os.getcwd(),'output', 'example'),
                                expandPaths=True,
                                debug=True)

In [8]:
###############################################################################
# Example 1: only select a specific modality (here MR) and make a call per image
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector, 
                        actionTag="example_1", 
                        generateCallable=my_function,
                        passOnlyURLs=True).do()

2021-12-06 14:56:25,885 [INFO] Starting action: PythonNaryBatchActionV2_example_1 (UID: 701ea6f7-c9b4-415c-8f87-8a6844a3daae) ...
2021-12-06 14:56:25,887 [INFO] Starting action: my_function (UID: e7aeb7b8-aee3-412c-ba4f-c249d93afecf) ...
output: {primaryInput:['pat1_TP1_MR.txt']}
2021-12-06 14:56:25,891 [INFO] Finished action: my_function (UID: e7aeb7b8-aee3-412c-ba4f-c249d93afecf) -> SUCCESS
2021-12-06 14:56:25,891 [INFO] Starting action: my_function (UID: 5981b38a-a63e-4ecb-b65f-2068871bfaa1) ...
output: {primaryInput:['pat1_TP2_MR.txt']}
2021-12-06 14:56:25,893 [INFO] Finished action: my_function (UID: 5981b38a-a63e-4ecb-b65f-2068871bfaa1) -> SUCCESS
2021-12-06 14:56:25,894 [INFO] Starting action: my_function (UID: 501d19b2-5c53-41cf-a21e-e8f7e6218044) ...
output: {primaryInput:['pat2_TP1_MR1.txt']}
2021-12-06 14:56:25,896 [INFO] Finished action: my_function (UID: 501d19b2-5c53-41cf-a21e-e8f7e6218044) -> SUCCESS
2021-12-06 14:56:25,896 [INFO] Starting action: my_function (UID: 498a4

In [9]:
###############################################################################
#Example 2: like example 1 but choose another variable name
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        actionTag="example_2", 
                        generateCallable=my_function, 
                        passOnlyURLs=True).do()

2021-12-06 14:56:28,874 [INFO] Starting action: PythonNaryBatchActionV2_example_2 (UID: 98c82a52-293d-4ad6-940e-c0bf242daa11) ...
2021-12-06 14:56:28,876 [INFO] Starting action: my_function (UID: 545cb2c5-4673-4454-b7f7-2d619420d91a) ...
output: {mr_images:['pat1_TP1_MR.txt']}
2021-12-06 14:56:28,879 [INFO] Finished action: my_function (UID: 545cb2c5-4673-4454-b7f7-2d619420d91a) -> SUCCESS
2021-12-06 14:56:28,879 [INFO] Starting action: my_function (UID: 4548dc40-6046-4ac8-8bcc-7bd2e30df991) ...
output: {mr_images:['pat1_TP2_MR.txt']}
2021-12-06 14:56:28,882 [INFO] Finished action: my_function (UID: 4548dc40-6046-4ac8-8bcc-7bd2e30df991) -> SUCCESS
2021-12-06 14:56:28,882 [INFO] Starting action: my_function (UID: 8544188e-d1a8-4d30-b1c2-66648c8490f5) ...
output: {mr_images:['pat2_TP1_MR1.txt']}
2021-12-06 14:56:28,884 [INFO] Finished action: my_function (UID: 8544188e-d1a8-4d30-b1c2-66648c8490f5) -> SUCCESS
2021-12-06 14:56:28,885 [INFO] Starting action: my_function (UID: af1e0265-f54c-

In [11]:
###############################################################################
#Example 3: select MR and CT images and call them pairwise (folding both sets)
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        # additionalInputSelectors = {'ct_images': ct_image_selector},
                        additionalInputSelectors = {'masks': mask_selector},
                        actionTag="example_3", 
                        generateCallable=my_function,
                        passOnlyURLs=True).do().tagSelector

2021-12-06 14:56:42,632 [INFO] Starting action: PythonNaryBatchActionV2_example_3 (UID: 7f36eae0-51cd-42df-874a-429e2f7d5ba3) ...
2021-12-06 14:56:42,634 [INFO] Successful actions: 2.
2021-12-06 14:56:42,635 [INFO] Skipped actions: 0.
2021-12-06 14:56:42,636 [INFO] Failed actions: 0.
2021-12-06 14:56:42,637 [INFO] Session finished. Feed me more...


ValueError: Input argument is invalid as it does not contain artefact instances or is None/empty. Input name: masks

In [ ]:
###############################################################################
#Example 3b:  select MR and CT images and call them pairwise (but only MRs and CTs of same patient and timepoint)
###############################################################################
with session:
    splitProperties=['case']
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors = {'additional': ct_image_selector},
                        # additionalInputSelectors = {'additional': mask_selector},
                        linker={'additional': CaseLinker(allowOnlyFullLinkage=False)+TimePointLinker(allowOnlyFullLinkage=False)},
                        splitter={'mr_images': KeyValueSplitter(*splitProperties), 'additional': CaseSplitter()},
                        actionTag="example_3b", 
                        generateCallable=my_function,
                        passOnlyURLs=True).do().tagSelector

In [ ]:
###############################################################################
#Example 4: select MR images and masks and call them casewise
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        # additionalInputSelectors ={'additional': mask_selector},
                        additionalInputSelectors = {'additional': ct_image_selector},
                        linker={'additional': CaseLinker(allowOnlyFullLinkage=False)+TimePointLinker(allowOnlyFullLinkage=False)},
                        splitter={'mr_images': CaseSplitter(), 'additional': CaseSplitter()},
                        actionTag="example_4", 
                        generateCallable=my_function, 
                        passOnlyURLs=True).do().tagSelector